In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import torch
import torch.nn as nn
from data import gen_data
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence,\
    pad_sequence, \
    pad_packed_sequence
from model import AE_rnn
import torch.optim as optim

In [51]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

data = gen_data(n_examples=int(1<<16),
                start_low=0, start_high=10,
                delta_low=1, delta_high=5,
                len_low=5, len_high=30)


class CustomData(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [9]:
def func(seq):
    print(type(seq))
    print(seq)
    seq = [torch.from_numpy(s).float() for s in seq]
    lengths = [len(s) for s in seq]
    
    padded_seq = pad_sequence(seq, batch_first=True).unsqueeze(-1) #this changes if use embeddings i.e. no unsqueeze but map to embeddings
    packed_padded_seq = pack_padded_sequence(padded_seq,
                                             lengths=lengths,
                                             batch_first=True,
                                             enforce_sorted=False)
    return packed_padded_seq
   

In [52]:
bidir          = True
dir            = 2 if bidir else 1 
batch_size     = 64
num_layers_enc = 3
hidden_dim_enc = 5
num_layers_dec = 2
hidden_dim_dec = 4
emb_dim        = 7
N_max          = 256


ds = CustomData(data) #dataset


ae = AE_rnn(batch_size     = batch_size,
            bidir          = True,
            num_layers_enc = num_layers_enc,
            hidden_dim_enc = hidden_dim_enc,
            num_layers_dec = num_layers_dec,
            hidden_dim_dec = num_layers_dec,
            emb_dim        = emb_dim,
            N_max          = N_max)

In [137]:
print(len(ds))

1000


In [ ]:
emb = nn.Embedding(num_embeddings=N_max,
                    embedding_dim=emb_dim)
for idx in range(0, len(ds), batch_size):
    seq = ds[idx:idx+batch_size]
    seq = [torch.from_numpy(s) for s in seq]
    lengths = torch.tensor([len(s) for s in seq])
    padded_seq = pad_sequence(seq, batch_first=True)
    packed_padded_seq = pack_padded_sequence(padded_seq,
                                lengths=lengths,#.tolist(),
                                batch_first=True,
                                enforce_sorted=False)
    packed_padded_seq_emb = emb(packed_padded_seq)
    padded_seq_emb = emb(padded_seq)
    packed_padded_emb_seq = pack_padded_sequence(padded_seq_emb,
                                lengths=lengths,#.tolist(),
                                batch_first=True,
                                enforce_sorted=False)
    print(packed_padded_emb_seq)
    


In [54]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Or "cuda:0" for the first GPU
else:
    device = torch.device("cpu")

#device = torch.device("cpu")
ae.to(device)


data_gpu = []
for idx in range(0, len(ds), batch_size):
    seq = ds[idx:idx+batch_size]
    seq = [torch.from_numpy(s).to(device) for s in seq]
    padded_seq = pad_sequence(seq, batch_first=True)
    lengths = [len(s) for s in seq]
    #lengths = torch.tensor([len(s) for s in seq]).to(device)
    data_gpu.append((padded_seq, lengths))


criterion = nn.CrossEntropyLoss()
#data_emb_device = [(edge_index.to(device), [bb.to(device) for bb in node_embs ]) for (edge_index, node_embs) in data_emb] 
optimizer = optim.Adam(ae.parameters(), lr=0.01)
epoch_num = 1000
for epoch in range(epoch_num):
    total_loss = 0
    total_items = 0
    j = -1
    for (padded_seq, lengths) in data_gpu:
        out = ae(padded_seq, lengths)
        loss = criterion(out.flatten(0).reshape(-1,N_max), padded_seq.flatten())
        total_loss  += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {total_loss:.5f}', end='\r')
    

KeyboardInterrupt: 

In [193]:
print(ae)

AE_rnn(
  (enc): LSTM(11, 5, num_layers=3, batch_first=True, bidirectional=True)
  (dec): LSTM(1, 7, num_layers=7, batch_first=True, bidirectional=True)
  (emb): Embedding(256, 11)
  (proj1): Linear(in_features=30, out_features=98, bias=True)
  (proj2): Linear(in_features=30, out_features=98, bias=True)
  (pred): Linear(in_features=14, out_features=256, bias=True)
)


In [ ]:
epoch_num = 1000
for epoch in range(epoch_num):
    total_loss = 0
    total_items = 0
    j = -1
    for graph in data_emb_device:
        j += 1
        edge_index, node_embs = graph
        for bb in node_embs:
            #bb_device = bb.to(device)
            out, embs = ae(bb)
            #print(a.shape)
            #print(out)
            #total_loss += unsupervised_loss.item()
            #loss = criterion(out, embs)
            loss = criterion(out, bb)
            #loss = criterion(output, initial_node_embeddings[j])
            # Backward pass and optimization
            total_loss  += loss.item()
            total_items += bb.shape[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Print or log the average loss for monitoring
    if epoch%10==0 or epoch==epoch_num-1:
        avg_loss = total_loss / len(data_emb)
        print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.5f}', end='\r')
    if epoch%100==0 or epoch==epoch_num-1:
        avg_loss = total_loss / len(data_emb)
        print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.5f}', end='\n')